In [ ]:
import torch

print (torch.cuda.is_available())
print (torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


True
Tesla T4


In [ ]:
import subprocess

from pathlib import Path
import spacy

import torch


dev_output_path = Path(r"/content/dev.spacy")
train_output_path = Path(r"/content/augmented_training_data.spacy")




base_config_path = Path(r"/content/base_config.cfg")


print("Initializing spaCy config...")
config_path = Path("./config.cfg")
if not config_path.exists():
    subprocess.run(
        ["python", "-m", "spacy", "init", "fill-config", base_config_path, "config.cfg"],
        check=True
    )


print("Validating spaCy data...")
print(f"Training data path: {train_output_path}")
result = subprocess.run(
    ["python", "-m", "spacy", "debug", "data",
        "config.cfg",
        "--paths.train", train_output_path,
        "--paths.dev", dev_output_path],
    capture_output=True, text=True
)


print("Starting spaCy training...")

output_dir = Path(r"/content/drive/MyDrive/CV-parsing/transformer")  # where the trained model will be saved
output_dir.mkdir(parents=True, exist_ok=True)

# Check for GPU availability
gpu_args = []
try:
    if torch.cuda.is_available():
        print(f"GPU detected: {torch.cuda.get_device_name(0)}. Training will use GPU.")
        gpu_args = ["--gpu-id", "0"]
    else:print("No GPU detected. Training will use CPU.")
except Exception as e:
    print(f"Could not check GPU status: {e}")

train_cmd = [
    "python", "-m", "spacy", "train", "config.cfg",
    "--output", str(output_dir),
    "--paths.train", str(train_output_path),
    "--paths.dev", str(dev_output_path)
] + gpu_args

train_result = subprocess.run(
    train_cmd,
    capture_output=True, text=True
)

if train_result.returncode == 0:
    print("Training completed successfully!")
    print(train_result.stdout)
else:
    print("Training failed!")
    print(train_result.stderr)
print("Training process finished.")

Initializing spaCy config...
Validating spaCy data...
Training data path: /content/augmented_training_data.spacy
Starting spaCy training...
GPU detected: Tesla T4. Training will use GPU.
Training completed successfully!
ℹ Saving to output directory:
/content/drive/MyDrive/CV-parsing/transformer
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        8699.02   1515.86    0.16    0.08    3.48    0.00
  0     200      241355.03  50152.51   12.72   36.95    7.68    0.13
  0     400       64003.50  16780.64   30.94   43.28   24.08    0.31
  0     600        6154.92  13366.38   36.01   64.38   25.00    0.36
  0     800        7322.43  12906